# Crop Growt Stages

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv("/content/LSTM_Model_all_crops.csv")

# Shuffle the DataFrame
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Extract NDVI values and replace values less than 0.1 with NaN
ndvi_df = df.iloc[:, 3:].mask(df.iloc[:, 3:] < 0.1)

# Interpolate NaN values
def interpolate_nan(row):
    for i in range(len(row)):
        if pd.isna(row[i]):
            prev_idx = i - 1
            while prev_idx >= 0 and pd.isna(row[prev_idx]):
                prev_idx -= 1
            next_idx = i + 1
            while next_idx < len(row) and pd.isna(row[next_idx]):
                next_idx += 1
            if prev_idx >= 0 and next_idx < len(row):
                interpolated_value = (row[prev_idx] + row[next_idx]) / 2
                row[i] = interpolated_value
            else:
                row[i] = 0.1
    return row

ndvi_data = ndvi_df.apply(interpolate_nan, axis=1)

# Generate list of dates starting from 2022-11-20 with a step of 5 days
start_date = pd.to_datetime('2022-11-20')
date_columns = [(start_date + pd.DateOffset(days=5 * i)).strftime('%Y-%m-%d') for i in range(len(ndvi_data.columns))]

# Create a mapping dictionary between NDVI columns and dates
column_mapping = dict(zip(ndvi_data.columns, date_columns))

# Rename columns using the mapping dictionary
ndvi_data.rename(columns=column_mapping, inplace=True)
ndvi_data['CropName'] = df['CropName']

# Define growth stages and their time differences for each crop
crop_growth_stages = {
    'Bajra': {'Sowing': 0, 'Emergence': 10, 'Flowering': 50, 'Maturity': 75, 'Harvest': 90},
    'Cotton': {'Sowing': 0, 'Emergence': 10, 'Flowering': 45, 'Boll formation': 75, 'Maturity': 105, 'Harvest': 120},
    'Gram': {'Sowing': 0, 'Emergence': 10, 'Flowering': 40, 'Pod-filling': 50, 'Maturity': 70, 'Harvest': 85},
    'Groundnut': {'Sowing': 0, 'Emergence': 10, 'Flowering': 40, 'Pod-filling': 70, 'Maturity': 85, 'Harvest': 115},
    'Maize': {'Sowing': 0, 'Emergence': 10, 'Tasseling': 40, 'Silking': 55, 'Maturity': 85, 'Harvest': 100},
    'Mustard': {'Sowing': 0, 'Emergence': 10, 'Flowering': 40, 'Maturity': 70, 'Harvest': 85},
    'Onion': {'Sowing': 0, 'Emergence': 10, 'Bulb-forming': 60, 'Maturity': 90, 'Harvest': 105},
    'Potato': {'Sowing': 0, 'Emergence': 10, 'Tuber initiation': 40, 'Tuber-bulking': 70, 'Maturity': 85, 'Harvest': 100},
    'Rice': {'Sowing': 0, 'Emergence': 10, 'Flowering': 40, 'Maturity': 70, 'Harvest': 85},
    'Soyabean': {'Sowing': 0, 'Emergence': 10, 'Reproductive growth': 25, 'Flowering': 40, 'Maturity': 70, 'Harvest': 85},
    'Wheat': {'Sowing': 0, 'Emergence': 10, 'Tillering': 40, 'Heading': 75, 'Maturity': 105, 'Harvest': 120}
}


# Define max NDVI stage for each crop
max_ndvi_stage = {
    'Bajra': 'Flowering',
    'Cotton': 'Boll formation',
    'Gram': 'Pod-filling',
    'Groundnut': 'Pod-filling',
    'Maize': 'Tasseling',
    'Mustard': 'Flowering',
    'Onion': 'Bulb-forming',
    'Potato': 'Tuber-bulking',
    'Rice': 'Flowering',
    'Soyabean': 'Reproductive growth',
    'Wheat': 'Heading'
}


def estimate_growth_stages(crop_data, crop_name):
    # Calculate the maximum NDVI value and its corresponding time
    max_ndvi_value = crop_data.max(axis=1)
    max_ndvi_time = crop_data.idxmax(axis=1)  # Assuming this gives the timestep index

    # Define growth stages and their time differences based on crop name
    growth_stages = crop_growth_stages[crop_name]

    # Estimate the timing of each growth stage
    estimated_growth_stages = {}

    # Append max NDVI stage and time to estimated growth stages
    max_ndvi_stage_name = max_ndvi_stage[crop_name]
    max_ndvi_stage_time = pd.to_datetime(max_ndvi_time.iloc[0])

    # Calculate time differences for stages before and after max NDVI stage
    for stage, days_difference in sorted(growth_stages.items(), key=lambda x: x[1]):
        if stage == max_ndvi_stage_name:
            estimated_growth_stages[stage] = max_ndvi_stage_time
        elif stage in crop_growth_stages[crop_name]:
            time_difference = crop_growth_stages[crop_name][max_ndvi_stage_name] - crop_growth_stages[crop_name][stage]
            if time_difference >= 0:
                estimated_time = max_ndvi_stage_time - pd.Timedelta(days=time_difference)
            else:
                estimated_time = max_ndvi_stage_time + pd.Timedelta(days=-time_difference)
            estimated_growth_stages[stage] = estimated_time

    return estimated_growth_stages


# Loop over each crop and estimate growth stages
crop_names = ndvi_data['CropName'].unique()
estimated_growth_stages_all_crops = {}
for crop_name in crop_names:
    crop_data = ndvi_data[ndvi_data['CropName'] == crop_name].drop(columns=['CropName'])
    estimated_growth_stages = estimate_growth_stages(crop_data, crop_name)
    estimated_growth_stages_all_crops[crop_name] = estimated_growth_stages


# Print the estimated growth stages for each crop
for crop_name, growth_stages in estimated_growth_stages_all_crops.items():
    print("Estimated growth stages for", crop_name)
    for stage, time in growth_stages.items():
        print(stage, ":", time)
    print()


Estimated growth stages for Maize
Sowing : 2022-12-25 00:00:00
Emergence : 2023-01-04 00:00:00
Tasseling : 2023-02-03 00:00:00
Silking : 2023-02-18 00:00:00
Maturity : 2023-03-20 00:00:00
Harvest : 2023-04-04 00:00:00

Estimated growth stages for Wheat
Sowing : 2022-11-20 00:00:00
Emergence : 2022-11-30 00:00:00
Tillering : 2022-12-30 00:00:00
Heading : 2023-02-03 00:00:00
Maturity : 2023-03-05 00:00:00
Harvest : 2023-03-20 00:00:00

Estimated growth stages for Groundnut
Sowing : 2022-11-15 00:00:00
Emergence : 2022-11-25 00:00:00
Flowering : 2022-12-25 00:00:00
Pod-filling : 2023-01-24 00:00:00
Maturity : 2023-02-08 00:00:00
Harvest : 2023-03-10 00:00:00

Estimated growth stages for Mustard
Sowing : 2023-03-05 00:00:00
Emergence : 2023-03-15 00:00:00
Flowering : 2023-04-14 00:00:00
Maturity : 2023-05-14 00:00:00
Harvest : 2023-05-29 00:00:00

Estimated growth stages for Potato
Sowing : 2022-12-05 00:00:00
Emergence : 2022-12-15 00:00:00
Tuber initiation : 2023-01-14 00:00:00
Tuber-bul